In [ ]:
import requests
API_KEY_FOOD = "%2BGZUDF8Y0GF2ilvaXCsR6HXBvfucD4KY0XwcmvTZ1l2zHfo7VZ7Gop7pGVM0DXX7RHSXjijQHxy8TAQ4K4vhVg%3D%3D"
page = "1"
perPage = "20"
url_food = f"https://api.odcloud.kr/api/15052408/v1/uddi:611c5470-ad94-49e8-8f72-973732c56304?page={page}&perPage={perPage}&serviceKey={API_KEY_FOOD}"
response_food = requests.get(url_food)
result_food = response_food.json()
food_list = result_food["data"]
food_list
# for i in range(len(result_food["data"])):
#     name = food_list[i]["업소명"]
#     food = food_list[i]["주취급음식"]
#     print(f"이름 : {name}, 음식 : {food}")

In [67]:
from datetime import datetime, timedelta
import pandas as pd
import random

df_gangnam = pd.read_csv("gangnam_places.csv")
df_gangnam.columns = df_gangnam.columns.str.strip()

category_durations = {
    "밥": timedelta(hours=1),
    "술집": timedelta(hours=3),
    "카페": timedelta(hours=2),
    "놀거리": timedelta(hours=1),
    "영화": timedelta(hours=3),
    "쇼핑": timedelta(hours=2)
}

meal_times = [(12, 13), (18, 19)]

def is_meal_time(current_time):
    return any(start <= current_time.hour < end for start, end in meal_times)

def map_to_category(row):
    name = str(row.get("이름", "")).strip()
    category_raw = str(row.get("카테고리", "")).strip()

    if "카페" in category_raw or "커피" in category_raw:
        return "카페"
    elif any(kw in category_raw for kw in ["술", "주점", "호프"]):
        return "술집"
    elif any(kw in category_raw for kw in ["밥", "음식", "한식", "탕", "국수", "김밥", "식당"]):
        return "밥"
    elif any(kw in category_raw for kw in ["영화", "극장", "CGV", "메가박스"]):
        return "영화"
    elif any(kw in category_raw for kw in ["백화점", "쇼핑", "몰", "아울렛"]):
        return "쇼핑"
    else:
        return "놀거리"

while True:
    area = input(str("지역을 입력하세요: "))

    if area == "강남":
        start_input = input(str("시간을 입력하세요: "))
        current_time = datetime.strptime(start_input, "%H:%M")

        end_of_day = datetime.strptime("23:59", "%H:%M")
        schedule = []
        df_pool = df_gangnam.copy()

        while current_time < end_of_day and not df_pool.empty:
            if is_meal_time(current_time):
                df_candidates = df_pool[df_pool["분류"].str.contains("음식점")]
                if df_candidates.empty:
                    df_candidates = df_pool
            else:
                df_candidates = df_pool

            place = df_candidates.sample(n=1).iloc[0]
            df_pool = df_pool.drop(index=place.name)

            name = place.get("이름")
            category = map_to_category(place)
            duration = category_durations.get(category, timedelta(hours=1))

            if current_time + duration > end_of_day:
                break

            schedule.append({
                "장소": name,
                "카테고리": category,
                "시작": current_time.strftime("%H:%M"),
                "종료": (current_time + duration).strftime("%H:%M")
            })

            current_time += duration

        print("\n오늘의 일정:")
        for item in schedule:
            print(f"{item['시작']} ~ {item['종료']} : {item['장소']} ({item['카테고리']})")
        break

    else:
        print("다시 입력.\n")


지역을 입력하세요:  신촌


다시 입력.



지역을 입력하세요:  강남
시간을 입력하세요:  16:00



오늘의 일정:
16:00 ~ 17:00 : 한류스타거리 (놀거리)
17:00 ~ 18:00 : 청담동명품거리 (놀거리)
18:00 ~ 19:00 : 세븐럭카지노 강남코엑스점 (놀거리)
19:00 ~ 20:00 : 서울선릉과정릉 (놀거리)
20:00 ~ 21:00 : 강남메디컬투어센터 (놀거리)
21:00 ~ 22:00 : 청담동명품거리 (놀거리)
22:00 ~ 23:00 : 대치동학원가 (놀거리)


In [27]:
print(df_gangnam.columns.tolist())

['이름', '카테고리', '주소', '전화번호', '지도URL', '분류']


In [69]:
import csv
import pandas as pd
import random

end_time = 24
time_list = []  

sequence_list = []

while(1):
    try: 
        input_time = int(input("약속 시간 입력 : "))
        if (input_time >= 10) and (input_time <= 23):
            start_time = input_time
            break
        else:
            print("10과 23 사이의 정수를 입력하세요")
    except ValueError:
        print("정수를 입력하세요")
##################################################################################################
if start_time <= 12:         # 점심식사 전 : time1 (= time_list[0])
    time1 = 12 - start_time   
else:
    time1 = 0
time_list.append(time1)

time2 = (18 - 13) if (start_time <= 12) else 0 if (start_time >= 18) else (18 - start_time)
time_list.append(time2)     # 점심식사 후 ~ 저녁식사 전 : time2 (=time_list[1])

if start_time > 18:
    time3 = 24 - start_time
else:
    time3 = 5
time_list.append(time3)     # 저녁식사 후 ~ 자정 : time3 (=time_list[2])
##################################################################################################
hour1 = ["meal", "play"]
hour2 = ["cafe", "shop"]
hour3 = ["drink", "movieshow"]

case_hour1 = "play"
case_hour2 = [["play", "play"], "cafe", "shop"]
case_hour3 = [["play", "play", "play"], ["play", hour2[random.randint(0,1)]], "movieshow", "drink"]
case_hour4 = [["play", "play", "play", "play"], ["play", "play", hour2[random.randint(0,1)]], ["play", hour3[random.randint(0,1)]]]
case_hour5 = [["play", "play", "play", "play", "play"], ["play", "play", "play", hour2[random.randint(0,1)]], ["play", "play", hour3[random.randint(0,1)]], ["play", "cafe", "shop"], [hour2[random.randint(0,1)], hour3[random.randint(0,1)]]]
##################################################################################################
for i in range(len(time_list)):

    if time_list[i] == 0:
        if (start_time != 12) and (start_time != 18):
            continue
    
    if time_list[i] == 1:
        case1 = case_hour1
        sequence_list.append(case1)

    if time_list[i] == 2:
        case2 = case_hour2[random.randint(0,2)]
        print(case2)
        
        # if type(case2) == "str":
        #     sequence_list += case2

        # else:
        #     sequence_list.extend(case2)

        if len(case2) == 2:
            sequence_list.extend(case2)

        elif len(case2) == 4:
            sequence_list.append(case2)

    if time_list[i] == 3:
        case3 = case_hour3[random.randint(0,3)]
        random.shuffle(case3)
        sequence_list.extend(case3)

    if time_list[i] == 4:
        case4 = case_hour4[random.randint(0,2)]
        random.shuffle(case4)
        sequence_list.extend(case4)

    if time_list[i] == 5:
        case5 = case_hour5[random.randint(0,4)]
        random.shuffle(case5)
        sequence_list.extend(case5)

    if (i != 2):
        sequence_list.append("meal")

print(sequence_list)

약속 시간 입력 :  16:00


정수를 입력하세요


약속 시간 입력 :  5


10과 23 사이의 정수를 입력하세요


약속 시간 입력 :  11


['play', 'meal', 'play', 'play', 'play', 'play', 'play', 'meal', 'drink', 'play', 'play']
